In [1]:
# Load all python libraries upfront
import numpy as np

In [2]:
# Sentiment analysis on the IMDB movie dataset (https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
# Below is the path to the downloaded labelled file

data_path = '/home/anilkag/data/imdb-movie-dataset/labeledTrainData.tsv'

### Read the data

In [3]:
def read_data(filename, SEPARATOR = '\t'):
    data_X = []
    data_y = []
    with open(filename) as fp:
        # Skip the header line (id \t sentiment \t review)
        fp.readline() 

        for line in fp:
            tokens = line.split(SEPARATOR)
            data_X.append( tokens[2] )
            data_y.append( int(tokens[1]) )
    return data_X, data_y

data_X, data_y = read_data(data_path)

### Review example

In [4]:
idx=0
print(data_y[idx])
print(data_X[idx])

1
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally s

In [5]:
# Preprocess using sklearn utilities
# Split into train/validation/test

In [6]:
# Write utility to compute all the performance metrics

In [7]:
# Train a classifier (note that this should to switch to something more fancier)

In [8]:
# Evaluate the basic classifier on test data

In [9]:
# Tune hyper-parameters on the validation data and then evaluate on test data

In [10]:
# Bonus tip : Sparsification of the classifier?